# Import utilities

In [1]:
from community_utils_3layers import *
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

import os

D:\Programs\Anaconda\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
%load_ext autoreload
%autoreload 2

----

# Read the files

##### Read the graph itself
'cora_edges.txt' -- the name of the file with the graph edge list
"cora_features" -- the name of the file with graph attributes data

In [3]:
'''Read the structural part of the graph'''
str_graph = nx.read_edgelist('data\datasets\cora\cora_edges.txt')

'''Assign weights to edges for structural graph'''
for edge in str_graph.edges():
    str_graph.add_edge(edge[0], edge[1], weight=1)

    
'''There are 1433 node features that are 1 or 0; and 1 categorical feature'''
feature_names = ["{}".format(ii) for ii in range(1433)]
column_names =  feature_names + ["subject"]
node_data = pd.read_csv(os.path.join(os.path.expanduser("~\PycharmProjects\modularity_2layers\data\datasets\cora"), 
                                     "cora_features"), sep='\t', header=None, names=column_names)

'''One hot encoding -- OHE -- for the categorical feature'''
OHE = OneHotEncoder()
categorical_feature = np.array(node_data['subject'])
least_features = OHE.fit_transform(categorical_feature.reshape(-1, 1))
least_features_df = pd.DataFrame(least_features.todense(), 
                                 columns = range(1433, 1433+7), 
                                 index = node_data.index)

'''Merge 1433 features with new features from OHE'''
final_data = pd.concat([node_data.iloc[:,:-1], least_features_df], axis=1)

'''Construct similarity matrix for identifying attributive graph'''
similarity_matrix = cosine_similarity(np.array(final_data))

'''Construct attributive graph'''
att_graph = nx.Graph()
att_graph.add_nodes_from(str_graph)  # to be sure that attributive graph has the same set of nodes as structural
for i1, node1 in enumerate(list(final_data.index)):
    for i2, node2 in enumerate(list(final_data.index)):
        if node1 != node2 and similarity_matrix[i1][i2] != 0:  # we do not need loops and 0-weighted edges
            att_graph.add_edge(str(node1), str(node2), weight=similarity_matrix[i1][i2])
            
'''There might be structurally isolated nodes with attributes. We should add these nodes to structural layer'''
str_graph.add_nodes_from(att_graph)

D:\Programs\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:395: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

D:\Programs\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:110: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for a

##### Read the partitions data if exist
'cora_METHOD_partit.txt' -- the name of the file with the best partition for the METHOD

In [ ]:
partition = {name: read_raw_file(file_name)
             for name, file_name in zip(['ef', 'sf' ,'lf'], ['data/partitions/cora/cora_ef_partit.txt', 
                                                             'data/partitions/cora/cora_sf_partit.txt', 
                                                             'data/partitions/cora/cora_lf_partit.txt'])}

Read the best partitions for the structural and attributive components that are used in LF

In [ ]:
partition_phase1 = {name: read_raw_file(file_name) for name, file_name 
                    in zip(['str', 'att'], ['data/partitions/cora/cora_par_first.txt', 
                                            'data/partitions/cora/cora_par_second.txt'])}

----

# Normalization

In [4]:
str_graph = normalize_to_fixed_weight_nx(str_graph, 1, show=True)
att_graph = normalize_to_fixed_weight_nx(att_graph, 1, show=True)

The whole weight has been equalt to 5277 . Now it is equal to 1.000000000000033
The whole weight has been equalt to 232149.5217694084 . Now it is equal to 0.9999999999982407


In [5]:
graph = {'str': str_graph, 'att': att_graph}
graph_names = graph.keys()

##### SAVE

In [6]:
save('data/datasets/cora/cora_str_graph.txt', list(str_graph.edges(data=True)))
save('data/datasets/cora/cora_str_graph.txt', list(str_graph.edges(data=True)))
save('data/datasets/cora/cora_att_graph.txt', list(att_graph.edges(data=True)))

----

# Define the best partitions for all the methods
If they do not yet exist, run that part and then run the whole notebook from the very beggining except this section.

In [7]:
simulations_number = 10
alphas_grid = [(val, 1-val) for val in np.linspace(0, 1, 21)]
alpha_order = {key: val for val, key in enumerate(graph_names)}
node_list = graph[take_first(graph)].nodes()

In [8]:
sf_partit = SF_louvain(graph, simulations_number, alphas_grid, alpha_order)

In [9]:
ef_partit = EF_louvain(graph, simulations_number, alphas_grid, alpha_order)

In [10]:
lf_partit, best_partit = LF_louvain(graph, simulations_number, alphas_grid, alpha_order)

Run this too if you have just defined the best partitions

In [11]:
partition = {'sf': sf_partit, 'ef': ef_partit, 'lf': lf_partit}

##### SAVE

In [14]:
save('data/partitions/cora/cora_sf_partit.txt', sf_partit)
save('data/partitions/cora/cora_ef_partit.txt', ef_partit)
save('data/partitions/cora/cora_lf_partit.txt', lf_partit)
save('data/partitions/cora/cora_par_first.txt', best_partit['str'])
save('data/partitions/cora/cora_par_second.txt', best_partit['att'])

----

# Composite modularity of the methods in terms of different objective functions

In [15]:
Q_sf = {method: compute_experiment('sf', partition[method], graph, node_list, alpha_order)
        for method in ['sf', 'ef', 'lf']}

Q_ef = {method: compute_experiment('ef', partition[method], graph, node_list, alpha_order)
        for method in ['sf', 'ef', 'lf']}

short_graph_layers_lf = {item[0]: late_fusion_transient_absorption(item[1], best_partit[item[0]]) 
                             for item in graph.items()}
Q_lf = {method: compute_experiment('lf', partition[method], short_graph_layers_lf, node_list, alpha_order)
        for method in ['sf', 'ef', 'lf']}

##### SAVE

In [16]:
save('data/metrics/cora/cora_Q_sf.txt', Q_sf)
save('data/metrics/cora/cora_Q_ef.txt', Q_ef)
save('data/metrics/cora/cora_Q_lf.txt', Q_lf)

##### PLOT

In [17]:
plot_dots_2d(Q_sf, '$Q_{SF}$', 'data/plots/cora/cora_sf.pdf')

In [18]:
plot_dots_2d(Q_ef, '$Q_{EF}$', 'data/plots/cora/cora_ef.pdf')

In [19]:
plot_dots_2d(Q_lf, '$Q_{LF}$', 'data/plots/cora/cora_lf.pdf')

----

## Delta

In [20]:
delta = {name: None for name in['sf', 'ef', 'lf']}
for name in ['sf', 'ef', 'lf']:
    delta[name] = delta_experiment(partition[name], str_graph, att_graph)

##### SAVE

In [21]:
save('data/metrics/cora/cora_delta.txt', delta)

##### PLOT

In [23]:
plot_dots_2d(delta, '$\Delta$', 'data/plots/cora/cora_delta.pdf')

----

## Theta

In [26]:
theta = {name: None for name in['sf', 'ef', 'lf']}
for name in ['sf', 'ef', 'lf']:
    theta[name] = theta_experiment(partition[name], best_partit, graph)

##### SAVE

In [27]:
save('data/metrics/cora/theta.txt', theta)

##### PLOT

In [28]:
plot_dots_2d(theta, '$\Theta$', 'data/plots/cora/cora_theta.pdf')

## Delta (G', C)

In [29]:
new_graph = {key: late_fusion_transient_absorption(graph[key], best_partit[key]) for key in graph.keys()}
delta_on_new_graph = {name: delta_experiment(partition[name], new_graph['str'], new_graph['att']) 
                      for name in ['sf', 'ef', 'lf']}

In [30]:
plot_dots_2d(delta_on_new_graph, '$\Delta`$', 'data/plots/cora/cora_delta_new.pdf')

# Leiden

In [31]:
simulations_number = 10
alphas_grid = [(val, 1-val) for val in np.linspace(0, 1, 21)]
alpha_order = {key: val for val, key in enumerate(graph_names)}
node_list = take_first(graph.values()).nodes()

In [32]:
graph_ig = {key: from_nx_to_ig_graph(graph[key], node_list) for key in graph.keys()}
    #  Calculate partitions
sf_partit = SF_leiden(graph, simulations_number, alphas_grid, alpha_order)
ef_partit = EF_leiden(graph, simulations_number, alphas_grid, alpha_order)
lf_partit, best_partit = LF_leiden(graph, simulations_number, alphas_grid, alpha_order)
    
#     best_partit = {k: val for k, val in zip(new_graph_nx.keys(), [str_partit, att_partit])}
partition = {'sf': sf_partit, 'ef': ef_partit, 'lf': lf_partit}
    
Q_sf = {method: compute_experiment('sf', partition[method], graph, node_list, alpha_order) for method in ['sf', 'ef', 'lf']}
Q_ef = {method: compute_experiment('ef', partition[method], graph, node_list, alpha_order) for method in ['sf', 'ef', 'lf']}

short_graph_layers_lf = {item[0]: late_fusion_transient_absorption(item[1], best_partit[item[0]]) 
                         for item in graph.items()}
Q_lf = {method: compute_experiment('lf', partition[method], short_graph_layers_lf, node_list, alpha_order) 
        for method in ['sf', 'ef', 'lf']}

In [33]:
plot_dots_2d(Q_sf, '$Q_{SF}$', 'data/plots/cora/cora_sf_leiden.pdf')

In [34]:
plot_dots_2d(Q_ef, '$Q_{EF}$', 'data/plots/cora/cora_ef_leiden.pdf')

In [35]:
plot_dots_2d(Q_lf, '$Q_{LF}$', 'data/plots/cora/cora_lf_leiden.pdf')

In [36]:
delta = {name: None for name in['sf', 'ef', 'lf']}
for name in ['sf', 'ef', 'lf']:
    delta[name] = delta_experiment(partition[name], graph['str'], graph['att'])

In [37]:
plot_dots_2d(delta, '$\Delta$', 'data/plots/cora/cora_delta_leiden.pdf')

In [38]:
theta = {name: None for name in['sf', 'ef', 'lf']}
for name in ['sf', 'ef', 'lf']:
    theta[name] = theta_experiment(partition[name], best_partit, graph)

In [39]:
plot_dots_2d(theta, '$\Theta$', 'data/plots/cora/cora_theta_leiden.pdf')

In [40]:
new_graph = {key: late_fusion_transient_absorption(graph[key], best_partit[key]) for key in graph.keys()}
delta_on_new_graph = {name: delta_experiment(partition[name], short_graph_layers_lf['str'], short_graph_layers_lf['att']) 
                      for name in ['sf', 'ef', 'lf']}

In [41]:
plot_dots_2d(delta_on_new_graph, '$\Delta`$', 'data/plots/cora/cora_delta_new_leiden.pdf')